# CodeTrans-T5-Small-ST-TF-Python

This notebook explores the CodeTrans model which is based on the T5 model's architecture. This particular model uses the T5-Small skeleton and is built for the purpose of a single task (ST), to generate code documentation for Python. This notebook will first use the model from HuggingFace and then attempt to fine-tune it (FT) on the [`code docstring corpus`](https://github.com/EdinburghNLP/code-docstring-corpus).

---